In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image
import seaborn as sns

import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorflow import keras
# from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,DepthwiseConv2D, MaxPooling2D, AvgPool2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
# from tensorflow.keras.layers import ReLU
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
 
# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths
from workbench.utils.utils import parse_model_name
from workbench.data.data import get_lemon_quality_dataset

#from dotenv import load_dotenv
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
#import deeplake

%load_ext autoreload



In [2]:
#%reload_ext autoreload
%autoreload

In [3]:
models_dir = initialize()

In [4]:
global model_name
#model_name = "efficientNetB0_1_96_c3_o3_keras"
#model_name = "shufflenetv2tinys_0.2_96_c3_o3_f4l1024"
model_name = "mobilenetv2_0.1_96_c3_o3_keras"
#model_name = "shufflenetv2tiny_0.1_96_c3_o3_f4l1024"
#model_name = "MobilenetV3small_1_96_c3_o3_keras"#, "MobilenetV3large_1_224_c3_o3_keras"# ,


In [5]:
base_model_name, alpha, resolution, channels, classes, variation = model_name.split("_")
alpha = float(alpha)
resolution = int(resolution)
classes = int(classes.strip("o"))
channels = int(channels.strip("c"))
PROJECT = "model_DB"

# just needed for INT 8 quantization:
BATCH_SIZE = 32
shuffle_seed = 1

In [6]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [7]:
model = keras.models.load_model(models_tf_path)

In [8]:
# view_model(model, tflite=True, build=True)

# Creating tflite Models

In [9]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter = tf.lite.TFLiteConverter.from_saved_model(models_path)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmppd7rd8ja\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmppd7rd8ja\assets


In [10]:
# Save the model.
with open(models_tflite_path, "wb") as f:
    f.write(tflite_model)

# tflite with INT8 quantization

In [11]:
dataset_path = Path.cwd().joinpath("datasets", "lemon_dataset")
dataset_path.exists()

True

In [12]:
train_ds, val_ds, test_ds, labels = get_lemon_quality_dataset(dataset_path, resolution, resolution, BATCH_SIZE, channels)

Color mode: rgb
Preparing training dataset...
Found 2021 files belonging to 3 classes.
Preparing validation dataset...
Found 252 files belonging to 3 classes.
Preparing test dataset...
Found 255 files belonging to 3 classes.
Class names: ['bad_quality', 'empty_background', 'good_quality']
Train: (TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
Normalize: True


In [13]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]
    
converter_opt = tf.lite.TFLiteConverter.from_keras_model(model)

# set the optimization flag
converter_opt.optimizations = [tf.lite.Optimize.DEFAULT]
# enforce integer only quantization
converter_opt.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_opt.inference_input_type = tf.uint8
converter_opt.inference_output_type = tf.uint8

# provide a representative dataset for quantization
converter_opt.representative_dataset = representative_data_gen

tflite_model_opt = converter_opt.convert()

# Save the model.
with open(models_tflite_opt_path, 'wb') as f:
  f.write(tflite_model_opt)
models_tflite_opt_path

INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpw59rzgex\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpw59rzgex\assets
d:\Miniconda\envs\tiny_cnn_6\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


WindowsPath('i:/tinyml/tiny_cnn/models/mobilenetv2_0.1_96_c3_o3_keras/mobilenetv2_0.1_96_c3_o3_keras_INT8.tflite')

# Profiling tflite models with MLTK profiler

In [14]:
mltk_model_stats = {}

In [15]:
mltk_model_stats["arhcitecture"] = model_name
mltk_model_stats["alpha"] = alpha
mltk_model_stats["img_res"] = resolution
mltk_model_stats["classes"] = classes
mltk_model_stats["channels"] = channels
mltk_model_stats["variation"] = variation


In [16]:
# MLTK profile model reads the mode from a path!

profiling_results = profile_model(str(models_tflite_path), accelerator=None, build=None)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 947328


In [17]:
mltk_model_stats["opt_RAM_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [18]:
profiling_results.get_summary(exclude_null=False, full_summary=True)
model_profile = profiling_results.to_dict()

In [19]:
mltk_model_stats["flash_model_size_b_mltk"] = model_profile["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_mltk"] = model_profile["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_mltk"] = model_profile["summary"]["ops"]
mltk_model_stats["macs_mltk"] = model_profile["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_mltk"] = model_profile["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_mltk"] = model_profile["summary"]["energy"]

In [20]:
layers_df = pd.DataFrame.from_dict(model_profile["layers"])
layers_df["name"] = layers_df["index"].map(str) + ": " + layers_df["opcode"]
layers_df.insert(2,"name" , layers_df.pop("name"))
layers_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,conv_2d,0: conv_2d,1050624,497664,4.600492e+06,8.359018e-04,"1x96x96x3,8x3x3x3,8",1x48x48x8,Padding:same stride:2x2 activation:relu6
1,1,depthwise_conv_2d,1: depthwise_conv_2d,387072,165888,2.522040e+06,4.679793e-04,"1x48x48x8,1x3x3x8,8",1x48x48x8,Multiplier:1 padding:same stride:1x1 activatio...
2,2,conv_2d,2: conv_2d,313344,147456,1.678246e+06,2.639363e-04,"1x48x48x8,8x1x1x8,8",1x48x48x8,Padding:same stride:1x1 activation:none
3,3,add,3: add,18432,0,1.550003e+06,3.223873e-04,"1x48x48x8,1x48x48x8",1x48x48x8,Activation:none
4,4,conv_2d,4: conv_2d,2101248,884736,1.073103e+07,2.051086e-03,"1x48x48x8,48x1x1x8,48",1x48x48x48,Padding:same stride:1x1 activation:relu6
...,...,...,...,...,...,...,...,...,...,...
65,65,conv_2d,65: conv_2d,55584,27648,9.964514e+04,3.828835e-05,"1x3x3x96,32x1x1x96,32",1x3x3x32,Padding:same stride:1x1 activation:none
66,66,conv_2d,66: conv_2d,771840,368640,1.500693e+06,7.942666e-05,"1x3x3x32,1280x1x1x32,1280",1x3x3x1280,Padding:valid stride:1x1 activation:relu6
67,67,mean,67: mean,0,0,1.732734e+06,1.093961e-05,"1x3x3x1280,2",1x1280,Type=reduceroptions
68,68,fully_connected,68: fully_connected,7680,3840,1.286945e+04,7.112388e-08,"1x1280,3x1280",1x3,Activation:none


In [21]:
layers_df_path = pathlib.Path.joinpath(models_dir, "layer_df_mltk.pkl")
layers_df.to_pickle(layers_df_path)

In [22]:
mltk_model_stats

{'arhcitecture': 'mobilenetv2_0.1_96_c3_o3_keras',
 'alpha': 0.1,
 'img_res': 96,
 'classes': 3,
 'channels': 3,
 'variation': 'keras',
 'opt_RAM_mltk': 947328,
 'flash_model_size_b_mltk': 370932,
 'RAM_runtime_memory_size_b_mltk': 943260,
 'OPS_mltk': 11391327,
 'macs_mltk': 4665408,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.008759774869950832}

In [23]:
# %%capture optimal_runtime_INT
profiling_results_INT8 = profile_model(str(models_tflite_opt_path), accelerator=None, build=False)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 271360


In [24]:
mltk_model_stats["opt_RAM_INT8_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [25]:
# from contextlib import redirect_stdout

# with open("runtime.txt", "w", encoding='utf-8') as f:
#     with redirect_stdout(f):
#         print(profile_model(str(models_tflite_opt_path), accelerator=None, build=False))

In [26]:
profiling_results_INT8.get_summary()

OrderedDict([('name', 'mobilenetv2_0.1_96_c3_o3_keras_INT8'),
             ('accelerator', 'None'),
             ('input_shape', '1x96x96x3'),
             ('input_dtype', 'uint8'),
             ('output_shape', '1x3'),
             ('output_dtype', 'uint8'),
             ('tflite_size', 220776),
             ('runtime_memory_size', 266576),
             ('ops', 11501931),
             ('macs', 4665408),
             ('n_layers', 72),
             ('n_unsupported_layers', 0),
             ('cpu_cycles', 51560335.30175781),
             ('cpu_utilization', 0.0),
             ('cpu_clock_rate', 78000000),
             ('energy', 0.008959035986997321),
             ('j_per_op', 7.789158174394648e-10),
             ('j_per_mac', 1.920311361192273e-09)])

In [27]:
model_profile_INT8 = profiling_results_INT8.to_dict()


In [28]:
mltk_model_stats["flash_model_size_b__INT8_mltk"] = model_profile_INT8["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_INT8_mltk"] = model_profile_INT8["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_INT8_mltk"] = model_profile_INT8["summary"]["ops"]
mltk_model_stats["macs_INT8_mltk"] = model_profile_INT8["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_INT8_mltk"] = model_profile_INT8["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_INT8_mltk"] = model_profile_INT8["summary"]["energy"]

In [29]:
mltk_model_stats

{'arhcitecture': 'mobilenetv2_0.1_96_c3_o3_keras',
 'alpha': 0.1,
 'img_res': 96,
 'classes': 3,
 'channels': 3,
 'variation': 'keras',
 'opt_RAM_mltk': 947328,
 'flash_model_size_b_mltk': 370932,
 'RAM_runtime_memory_size_b_mltk': 943260,
 'OPS_mltk': 11391327,
 'macs_mltk': 4665408,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.008759774869950832,
 'opt_RAM_INT8_mltk': 271360,
 'flash_model_size_b__INT8_mltk': 220776,
 'RAM_runtime_memory_size_b_INT8_mltk': 266576,
 'OPS_INT8_mltk': 11501931,
 'macs_INT8_mltk': 4665408,
 'n_unsupported_layers_INT8_mltk': 0,
 'energy_INT8_mltk': 0.008959035986997321}

In [30]:
layers_INT8_df = pd.DataFrame.from_dict(model_profile_INT8["layers"])
layers_INT8_df["name"] = layers_INT8_df["index"].map(str) + ": " + layers_INT8_df["opcode"]
layers_INT8_df.insert(2,"name" , layers_INT8_df.pop("name"))
layers_INT8_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,quantize,0: quantize,110592,0,9.957829e+05,1.988387e-04,1x96x96x3,1x96x96x3,Type=none
1,1,conv_2d,1: conv_2d,1050624,497664,4.600492e+06,8.359018e-04,"1x96x96x3,8x3x3x3,8",1x48x48x8,Padding:same stride:2x2 activation:relu6
2,2,depthwise_conv_2d,2: depthwise_conv_2d,387072,165888,2.522040e+06,4.679793e-04,"1x48x48x8,1x3x3x8,8",1x48x48x8,Multiplier:1 padding:same stride:1x1 activatio...
3,3,conv_2d,3: conv_2d,313344,147456,1.678246e+06,2.639363e-04,"1x48x48x8,8x1x1x8,8",1x48x48x8,Padding:same stride:1x1 activation:none
4,4,add,4: add,18432,0,1.550003e+06,3.223873e-04,"1x48x48x8,1x48x48x8",1x48x48x8,Activation:none
...,...,...,...,...,...,...,...,...,...,...
67,67,conv_2d,67: conv_2d,771840,368640,1.500693e+06,7.942666e-05,"1x3x3x32,1280x1x1x32,1280",1x3x3x1280,Padding:valid stride:1x1 activation:relu6
68,68,mean,68: mean,0,0,1.732734e+06,1.093961e-05,"1x3x3x1280,2",1x1280,Type=reduceroptions
69,69,fully_connected,69: fully_connected,7680,3840,1.286945e+04,7.112388e-08,"1x1280,3x1280",1x3,Activation:none
70,70,softmax,70: softmax,15,0,3.838418e+03,1.652612e-08,1x3,1x3,Type=softmaxoptions


In [31]:
layers_df_INT8_path = pathlib.Path.joinpath(models_dir, "layer_df_INT8_mltk.pkl")
layers_INT8_df.to_pickle(layers_df_INT8_path)

In [32]:
#layers_INT8_df.info()

In [33]:
api = wandb.Api()

# Generate run ids
run_id = wandb.util.generate_id()

wandb.init(
    project=PROJECT,
    entity="susbrock",
    id=run_id,
    resume=True)
config = wandb.config
config.id = run_id
config.update(mltk_model_stats)

wandb.finish()



ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: susbrock. Use `wandb login --relogin` to force relogin
